In [1]:
import pandas as pd
import string
import unidecode
import time
import os

df = pd.read_csv(r"br-capes-bolsistas-uab.csv", sep=';', encoding='latin-1')


### Testando a consulta bolsista ano zero

In [3]:
anos = [2014,2015,2013,2016]
achou = False
 
for ano in anos:
    for i, info in enumerate(df['AN_REFERENCIA']):
        if ano == info:
            index = i
            if df.iloc[index, 0] == 'JOSE QUERGINALDO BEZERRA': achou = True #2013
            if df.iloc[index, 0] == 'CAROLINE ALCANTARA DUARTE': achou = True #2014
            if df.iloc[index, 0] == 'CICERO SARAIVA SOBRINHO': achou = True #2015
            if df.iloc[index, 0] == 'ALEXANDRE RIBEIRO NETO': achou = True #2016     
            
        
print(achou)      

True


### Codificar Nomes

* O objetivo principal desse teste é ver se o programa funcionaria com cada nome que tem na tabela. Então se por acaso der algum erro, eu saberei que tem algo a mais para tratar.

* Essa parte demora para rodar, se for cancelada no meio, irá indicar incorretamente um erro

In [ ]:
def editar_texto(nome):
    
    nome = nome.upper() # Transformando para Maiúsculo
    nome = unidecode.unidecode(nome) # Retirando Acentos
    nomes_vetor = nome.split() # Colocando cada palavra do nome com se fosse um item de um vetor
    vetor_palavras = []

    for palavra in nomes_vetor:
        for letra in palavra:
            if letra.isalnum() == False and letra != " ": # verificando se há caracter especial
                palavra = palavra.replace(letra, "") # Trocando o  caracter especial por "nada"
        vetor_palavras.append(palavra) 

    return "".join(vetor_palavras)  # Juntando os itens do vetor para formar uma palavra



def codificador(nome): 
    
    nome = nome[-1:] + nome[1:-1] + nome[:1]  # Fazendo o swap da primeira com a última letra

    if len(nome) >= 4:                       # Se a palavra tiver menos de 4 letras só o swap é necessário, senao, temos que inverter
        nome = nome[::-1]                   # Invertendo a string

    vet_nome = []
    alfabeto = string.ascii_uppercase #Criando um vetor com o alfabeto MAIUSCULO
    
    # For´s para mudar letra por letra
    for j in range(len(nome)):
        for i in range(len(alfabeto)):
            if nome[j] == alfabeto[i]:
                if nome[j] == 'Z':      #Z é especial pois é o ultimo do vetor, então não existira um próximo
                    letra_new = nome[j].replace('Z', 'A')
                    vet_nome.append(letra_new)
                else:   
                    letra_new = nome[j].replace(nome[j], alfabeto[i+1])  # Mudando para o próximo carácter
                    vet_nome.append(letra_new )
   
    return "".join(vet_nome)  #vet_nome = ['a','b','c']  -> .join -> vet_nome = ['abc']


for name in df['NM_BOLSISTA']:
    try:
        
        nome_completo = name.split() # Separando cada nome digitado como se fosse um item do vetor, para facilitar o tratamento

        nomes_vetor = []
        nome_editado = []

        for nome in nome_completo:
            nome_editado.append(editar_texto(nome)) # Exemplo nome_editado = ['JOAO', 'VITOR']

                
        # For que acha o bolsista mesmo se a pessoa não digitar o nome dele em ordem ou completo.
        for nome_bolsista in df['NM_BOLSISTA']:  
            iguais = set(nome_editado).intersection(nome_bolsista.split())  # Verifica quais itens dos vetores são iguais.
            if len(iguais) == len(nome_editado):          # Se todos itens forem iguais, então os vetores tem tamanho iguais,
                nome_bolsista_completo = nome_bolsista   # isso signififca que encontramos a pessoa procurada.
                break


        for nome in nome_bolsista_completo.split(): # Codificando os nomes
            nomes_vetor.append(codificador(nome) + " ") # O espaço é para garantir que na junção do vetor os nomes fiquem separados
            nome_pronto_codificado = "".join(nomes_vetor)


            
    except:
            print(f"Não encontramos {nome_completo} na nossa base de dados. Por favor, tente novamente!")

###  Ranking de bolsas

* Com o comando abaixo, vemos todos os valores disponíveis na coluna e bolsa, então ja sabemos os maiores e os menores

In [12]:
df['VL_BOLSISTA_PAGAMENTO'].value_counts()

765     51247
1300    18549
1100     7501
1400     2048
1500      670
Name: VL_BOLSISTA_PAGAMENTO, dtype: int64